In [47]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 64569,
  "iopub_port": 64571,
  "stdin_port": 64573,
  "control_port": 64576,
  "hb_port": 64579,
  "ip": "127.0.0.1",
  "key": "d0551bb2-e3057811c6c4230f5be764aa",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-17aad4ac-2972-42b5-9845-ce7939879c54.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


In [50]:

@dataclass
class AntagonisticGameOptimizer:
    A: float
    B: float
    c: float
    d: float
    e: float

    def H(self, x: float, y: float) -> float:
        return self.A*x**2 + self.B*y**2 + self.c*x*y + self.d*x + self.e*y
    
    def __post_init__(self):
        if 2*self.A >= 0 and 2*self.B <= 0:
            raise ValueError("This game is not solvable by this solver")
        
    def analytical_solution(self) -> Tuple[float, float, float]:
        x = (self.c*self.e - 2*self.B*self.d) / (4*self.B*self.A - self.c**2)
        y = -(self.c*x + self.e) / (2*self.B)
        x = x if x <= -self.e/self.c else 0
        y = y if y >= -self.d/self.c else 0
        solution = self.H(x, y)
        print(f"Analytical optimal solution: x={x:.3f}, y={y:.3f}, H(x,y)={solution:.2f}")
        return x,y,solution
    
    @staticmethod
    def minmax(matrix) -> Tuple[int, int]:
        max_points = np.amax(matrix, axis=0)
        column = np.argmin(max_points)
        row = np.argmax(matrix[:,column])
        #print(f"Minimax position: {row, column}")
        return row, column
        
    @staticmethod
    def maxmin(matrix) -> Tuple[int, int]:
        min_points = np.amin(matrix, axis=1)
        #print(f"Maxmin points: {min_points}")
        row = np.argmax(min_points)
        column = np.argmin(matrix[row])
        #print(f"Maximin position: {row, column}")
        return row, column
    
    @classmethod
    def has_point(cls, matrix) -> bool:
        return cls.maxmin(matrix) == cls.minmax(matrix)
        
    def step(self, matrix_size: int) -> Tuple[float, float, float]:
        M = np.empty([matrix_size + 1, matrix_size + 1])
        for i in range(0, matrix_size + 1):
            for j in range(0, matrix_size + 1):
                M[i][j] = self.H(float(i) / matrix_size, float(j) / matrix_size)
        df = pd.DataFrame(M)
        print(f"M table with N={matrix_size}")
        display(df)
        if not self.has_point(M):
            print("No saddle point. Use Brown-Robinson for decisions...")
            opt = BrownRobinsonOptimizer(M, max_steps=10000, verbose=False)
            strategies, _ = opt.fit(0.001)
            x = strategies.A
            y = strategies.B
            H = x @ M @ y.T
            print(f"X: {x}, Y: {y}")
            x = np.argmax(x) / matrix_size
            y = np.argmax(y) / matrix_size
        else:
            mins = np.amin(M, axis=1)
            print(f"Minima for M: {mins}")
            mins_pos = np.argmin(M, axis=1)
            print(f"Minima positions for M: {mins_pos}")
            H = max_min = np.amax(mins)
            max_min_pos = np.argmax(mins)
            print(f"Maximin positions for M: {max_min_pos}")
            y = mins_pos[max_min_pos] / matrix_size
            x = max_min_pos / matrix_size

        print(f"N: {matrix_size},\tx={x:.2f},\ty={y:.2f},\tH={H:.2f}")
        return x,y,H
    
    def fit(self, iterations: int=10) -> Tuple[float, float, float]:
        states = []
        for N in range(2, iterations+1):
            states.append(self.step(N))
            print("============")
        states = sorted(states, key=lambda x: x[2], reverse=True)
        print(f"Best solution: x={states[0][0]:.3f}, y={states[0][1]:.3f}, H={states[0][2]:.3f}")
        return states[0]
        

In [51]:
opt = AntagonisticGameOptimizer(A=-3, B=3/2, c=18/5, d=-18/50, e=-72/25)
#opt = AntagonisticGameOptimizer(A=-5, B=9/2, c=15, d=-9/2, e=-9)
opt = AntagonisticGameOptimizer(A=-6, B=32/5, c=16, d=-16/5, e=-64/5)
opt.fit(iterations=10)
_ = opt.analytical_solution()

M table with N=2


,0,1,2
0,0.0,-4.8,-6.4
1,-3.1,-3.9,-1.5
2,-9.2,-6.0,0.4


Minima for M: [-6.4 -3.9 -9.2]
Minima positions for M: [2 1 0]
Maximin positions for M: 1
N: 2,	x=0.50,	y=0.50,	H=-3.90
M table with N=3


,0,1,2,3
0,0.000000,-3.555556,-5.688889,-6.400000
1,-1.733333,-3.511111,-3.866667,-2.800000
2,-4.800000,-4.800000,-3.377778,-0.533333
3,-9.200000,-7.422222,-4.222222,0.400000


No saddle point. Use Brown-Robinson for decisions...
X: [2.53549696e-04 8.09837728e-01 1.89908722e-01 0.00000000e+00], Y: [1.26774848e-04 2.75101420e-01 7.24391481e-01 3.80324544e-04]
N: 3,	x=0.33,	y=0.67,	H=-3.77
M table with N=4


...

Minima for M: [-6.4     -5.04375 -4.175   -3.84375 -4.      -4.64375 -5.775   -7.39375
 -9.2    ]
Minima positions for M: [8 7 6 4 3 2 0 0 0]
Maximin positions for M: 3
N: 8,	x=0.38,	y=0.50,	H=-3.84
M table with N=9


,0,1,2,3,4,5,6,7,8,9
0,0.000000,-1.343210,-2.528395,-3.555556,-4.424691,-5.135802,-5.688889,-6.083951,-6.320988,-6.400000
1,-0.429630,-1.575309,-2.562963,-3.392593,-4.064198,-4.577778,-4.933333,-5.130864,-5.170370,-5.051852
2,-1.007407,-1.955556,-2.745679,-3.377778,-3.851852,-4.167901,-4.325926,-4.325926,-4.167901,-3.851852
3,-1.733333,-2.483951,-3.076543,-3.511111,-3.787654,-3.906173,-3.866667,-3.669136,-3.313580,-2.800000
4,-2.607407,-3.160494,-3.555556,-3.792593,-3.871605,-3.792593,-3.555556,-3.160494,-2.607407,-1.896296
5,-3.629630,-3.985185,-4.182716,-4.222222,-4.103704,-3.827160,-3.392593,-2.800000,-2.049383,-1.140741
6,-4.800000,-4.958025,-4.958025,-4.800000,-4.483951,-4.009877,-3.377778,-2.587654,-1.639506,-0.533333
7,-6.118519,-6.079012,-5.881481,-5.525926,-5.012346,-4.340741,-3.511111,-2.523457,-1.377778,-0.074074
8,-7.585185,-7.348148,-6.953086,-6.400000,-5.688889,-4.819753,-3.792593,-2.607407,-1.264198,0.237037
9,-9.200000,-8.765432,-8.172840,-7.422222,-6.513580,-5.446914,-4.222222,-2.839506,-1.298765,0.400000


No saddle point. Use Brown-Robinson for decisions...
X: [2.19876869e-04 0.00000000e+00 0.00000000e+00 4.00285840e-01
 5.98614776e-01 4.39753738e-04 4.39753738e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00], Y: [1.09938434e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.81794195e-01 4.17656113e-01 1.09938434e-04 1.09938434e-04
 0.00000000e+00 2.19876869e-04]
N: 9,	x=0.44,	y=0.44,	H=-3.84
M table with N=10


,0,1,2,3,4,5,6,7,8,9,10
0,0.00,-1.216,-2.304,-3.264,-4.096,-4.80,-5.376,-5.824,-6.144,-6.336,-6.400000e+00
1,-0.38,-1.436,-2.364,-3.164,-3.836,-4.38,-4.796,-5.084,-5.244,-5.276,-5.180000e+00
2,-0.88,-1.776,-2.544,-3.184,-3.696,-4.08,-4.336,-4.464,-4.464,-4.336,-4.080000e+00
3,-1.50,-2.236,-2.844,-3.324,-3.676,-3.90,-3.996,-3.964,-3.804,-3.516,-3.100000e+00
4,-2.24,-2.816,-3.264,-3.584,-3.776,-3.84,-3.776,-3.584,-3.264,-2.816,-2.240000e+00
5,-3.10,-3.516,-3.804,-3.964,-3.996,-3.90,-3.676,-3.324,-2.844,-2.236,-1.500000e+00
6,-4.08,-4.336,-4.464,-4.464,-4.336,-4.08,-3.696,-3.184,-2.544,-1.776,-8.800000e-01
7,-5.18,-5.276,-5.244,-5.084,-4.796,-4.38,-3.836,-3.164,-2.364,-1.436,-3.800000e-01
8,-6.40,-6.336,-6.144,-5.824,-5.376,-4.80,-4.096,-3.264,-2.304,-1.216,-1.776357e-15
9,-7.74,-7.516,-7.164,-6.684,-6.076,-5.34,-4.476,-3.484,-2.364,-1.116,2.600000e-01


Minima for M: [-6.4   -5.276 -4.464 -3.996 -3.84  -3.996 -4.464 -5.276 -6.4   -7.74
 -9.2  ]
Minima positions for M: [10  9  7  6  5  4  2  1  0  0  0]
Maximin positions for M: 4
N: 10,	x=0.40,	y=0.50,	H=-3.84
Best solution: x=0.333, y=0.667, H=-3.769
Analytical optimal solution: x=0.400, y=0.500, H(x,y)=-3.84
